![Add a relevant banner image here](path_to_image)

# Project Title

## Overview

Short project description. Your bottom line up front (BLUF) insights.

## Business Understanding

Text here

## Data Understanding

Text here

In [2]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.Collecting pyspark
  Using cached pyspark-4.0.1.tar.gz (434.2 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached py4j-0.10.9.9-py2.py3-none-any.whl.metadata (1.3 kB)
Using cached py4j-0.10.9.9-py2.py3-none-any.whl (203 kB)
  Created wheel for pyspark: filename=pyspark-4.0.1-py2.py3-none-any.whl size=434813901 sha256=329e34e4aa9b27564dcb85fa215617a62ddd7dd63adec809aa20a51cddc68281
  Stored in directory: c:\users\markh\appdata\local\pip\cache\wheels\10\e6\6b\c50eb601fa827dd56a5272db5d5db360e559e527a80a665b1d
Successfully built pyspark

   ---------------------------------------- 0/2 [py4j]
   ---------------------------------------

In [3]:
# Load relevant imports here
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import gc
import csv
from datetime import time
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import randint, reciprocal
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import min, max, col


In [4]:
spark = SparkSession.builder.master('capstone_proj_1').getOrCreate()

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [5]:
df_all_data = spark.read.csv('Data/US_Accidents_March23.csv')
print(df_all_data.describe())
print(df_all_data.head())
print(df_all_data.columns)

NameError: name 'spark' is not defined

In [ ]:
df_severity_by_state = pd.crosstab(df_all_data['Severity'], df_all_data['State'])
df_severity_by_state.head()

In [ ]:
# chart of accident severity by state
states = df_severity_by_state.columns
sev1 = df_severity_by_state.iloc[0]
sev2 = df_severity_by_state.iloc[1]
sev3 = df_severity_by_state.iloc[2]
sev4 = ct_sev4_by_state = df_severity_by_state.iloc[3]

plt.figure(figsize=(12, 7))
plt.bar(states, sev1)
plt.bar(states, sev2, bottom=sev1)
plt.bar(states, sev3, bottom=sev1+sev2)
plt.bar(states, sev4, bottom=sev1+sev2+sev3)
plt.xlabel("State")
plt.xticks(rotation=60)
plt.ylabel("Number of Accidents")
plt.legend(["Severity 1", "Severity 2", "Severity 3", "Severity 4"])
plt.title("Accident Severity by State (2016 - 2023)")
plt.show()

In [ ]:
# chart of accident severity by state without sev2 (sev 2 >> than the others so obscures sev 1, 3, 4 above)
states = df_severity_by_state.columns
sev1 = df_severity_by_state.iloc[0]
sev3 = df_severity_by_state.iloc[2]
sev4 = ct_sev4_by_state = df_severity_by_state.iloc[3]

plt.figure(figsize=(12, 7))
plt.bar(states, sev1)
plt.bar(states, sev3)
plt.bar(states, sev4)
plt.xlabel("State")
plt.xticks(rotation=60)
plt.ylabel("Number of Accidents")
plt.legend(["Severity 1", "Severity 3", "Severity 4"])
plt.title("Accident Severity by State, Excluding Severity 2 (2016 - 2023)")
plt.show()

In [ ]:
df_sev_by_crossing = pd.crosstab(df_all_data['Severity'], df_all_data['Crossing'])
df_sev_by_crossing = df_sev_by_crossing.rename(columns={False: "No", True: "Yes"})
df_sev_by_crossing.head()

In [ ]:
# charts of accident severity by bump, traffic calming, roundabout
# chart of accident severity by state
crossing = df_sev_by_crossing.columns
cr1 = df_sev_by_crossing.iloc[0]
cr2 = df_sev_by_crossing.iloc[1]
cr3 = df_sev_by_crossing.iloc[2]
cr4 = df_sev_by_crossing.iloc[3]

plt.figure(figsize=(10, 7))
plt.bar(crossing, cr1)
plt.bar(crossing, cr2)
plt.bar(crossing, cr3)
plt.bar(crossing, cr4)
plt.xticks(rotation=60)
plt.xlabel("Nearby Crossing")
plt.ylabel("Number of Accidents")
plt.legend(["Severity 1", "Severity 2", "Severity 3", "Severity 4"])
plt.title("Accident Severity by Proximity to Crossing, (2016 - 2023)")
plt.show()

In [ ]:
print(f"Count of Wind Direction Entries: {df_all_data['Wind_Direction'].nunique()}")

## Data Preparation

### Data Selection

Based on my exploration of the data, I'm dropping the following fields from the dataset for the following reasons:

- Source: contains information that has no relationship to causes and effects of accidents
- Start_Lat, Start_Lng, End_Lat, End_Lng, Street, City, County, State, Zipcode, Country: I'm going to model factors that influence accident severity independent of location.
    - I'll drop Zipcode and State data cleaning and construction since they'll be used for imputation and creating a new field
- Start_Time, End_Time: as with location features, I'm dropping these because I'll model factors independent of time
    - I'll drop Start_Time after imputation
- Description: unstructured text that will not give meaningful results with the planned modeling
- Timezone: duplicates zip/state with less precision
- Street: this field is not standardized and will introduce noise to the modeling
- Country: all data is from the United States so this field is redundant
- Airport_Code: doesn't provide germane information-the exact location where weather conditions are reported is not a variable that can be adjusted
- Weather_Timestamp: not related to the conditions of the accidents in any way
- Wind_Direction: too many unique values; values are also not related to travel directions so it's unlikely they'll  produce clear/actionable conclusions


In [ ]:
df_refined = df_all_data.drop(['ID', 'Source', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Description',
                                'Timezone', 'Street', 'City', 'County', 'Country', 'Airport_Code', 'Weather_Timestamp', 'Wind_Direction'], axis=1)

print(df_refined.describe())
print(df_refined.info())
print(df_refined.columns)

In [ ]:
del df_all_data
gc.collect()

### Data Cleaning

##### Missing Values

I've managed missing values in the code blocks below. Here's a brief explanation of my approch to each column:

- Temperature, Wind_Chill, Humidity, Pressure, Visibility: imputed based on the mean temp of other accident entries sharing the same day and zip code (or state if there are none in the zip code)
- Precipitation, Wind_Speed: assumed NaN indicates no precipitation/wind and replaced NaN with zero
- Sunrise_Sunset: imputed based on
- Civil_Twilight: imputed based on
- Nautical_Twilight: imputed based on
- Astronomical_Twilight: imputed based on

In [ ]:
df_refined['Start_Time'] = pd.to_datetime(df_refined['Start_Time'], yearfirst=True, format='mixed')
df_refined['End_Time'] = pd.to_datetime(df_refined['End_Time'], yearfirst=True, format='mixed')
df_refined['Acc_date'] = df_refined['Start_Time'].dt.date
df_refined['Acc_time'] = df_refined['Start_Time'].dt.time

In [ ]:
# replacing NaN values in the temperature column

def fill_missing_temp(df, date='Acc_date', zip='Zipcode', state='State', temp='Temperature(F)'):
    """
    Replaces NaN humidity values with the mean humidity of entries
    with the same date and zip code. If no match exists with date and county,
    uses date and state.
    """
    # Step 1: mean humidity for each date-zip combination
    df_temp_fill = df_refined.copy()
    
    temp_means_zip = df_temp_fill.groupby([date, zip])[temp].transform('mean')
    
    # fill NaN values with the date-zip group mean
    df_temp_fill[temp] = df_temp_fill[temp].fillna(temp_means_zip)
    
    # Step 2: date-state combination for remaining NaNs
    if df_temp_fill[temp].isna().any():
        remaining = df_temp_fill[temp].isna().sum()
        print(f"Info: {remaining} temperature entries still missing after date-zip fill.")
        print("Filling remaining with date-state mean.")
        
        temp_means_state = df_temp_fill.groupby([date, state])[temp].transform('mean')
        df_temp_fill[temp] = df_temp_fill[temp].fillna(temp_means_state)
    
    # Step 3: if any NaNs still remain, fill with overall mean as last resort
    if df_temp_fill[temp].isna().any():
        remaining = df_temp_fill[temp].isna().sum()
        print(f"Warning: {remaining} temperature entries still missing after date-state fill.")
        print("Filling remaining with overall mean as last resort.")
        df_temp_fill[temp] = df_temp_fill[temp].fillna(df_temp_fill[temp].mean())
    
    return df_temp_fill

df_temp_fill = fill_missing_temp(df_refined)

In [ ]:
# verifying NaNs have been replaced
df_temp_fill['Temperature(F)'].isna().sum()

In [ ]:
# writing NaN-free column back to the original dataframe
df_refined['Temperature(F)'] = df_temp_fill['Temperature(F)']
print(f"Entries with NaN Temperature: {df_refined['Temperature(F)'].isna().sum()}")

In [ ]:
# deleting working df to free up memory
del df_temp_fill
gc.collect()

In [ ]:
# replacing NaN values in the wind chill column

def fill_missing_windchill(df, date='Acc_date', zip='Zipcode', state='State', windchill='Wind_Chill(F)'):
    """
    Replaces NaN wind chill values with the mean wind chill of entries
    with the same date and zip code. If no match exists with date and county,
    uses date and state.
    """
    # Step 1: mean temperature for each date-zip combination
    df_windchill_fill = df_refined.copy()
    
    windchill_means_zip = df_windchill_fill.groupby([date, zip])[windchill].transform('mean')
    
    # fill NaN values with the date-zip group mean
    df_windchill_fill[windchill] = df_windchill_fill[windchill].fillna(windchill_means_zip)
    
    # Step 2: date-state combination for remaining NaNs
    if df_windchill_fill[windchill].isna().any():
        remaining = df_windchill_fill[windchill].isna().sum()
        print(f"Info: {remaining} wind chills still missing after date-zip fill.")
        print("Filling remaining with date-state mean.")
        
        windchill_means_state = df_windchill_fill.groupby([date, state])[windchill].transform('mean')
        df_windchill_fill[windchill] = df_windchill_fill[windchill].fillna(windchill_means_state)
    
    # Step 3: if any NaNs still remain, fill with overall mean as last resort
    if df_windchill_fill[windchill].isna().any():
        remaining = df_windchill_fill[windchill].isna().sum()
        print(f"Warning: {remaining} wind chills still missing after date-state fill.")
        print("Filling remaining with overall mean as last resort.")
        df_windchill_fill[windchill] = df_windchill_fill[windchill].fillna(df_windchill_fill[windchill].mean())
    
    return df_windchill_fill

df_windchill_fill = fill_missing_windchill(df_refined)

In [ ]:
# verifying NaNs have been replaced in the working df
df_windchill_fill['Wind_Chill(F)'].isna().sum()

In [ ]:
# writing NaN-free column back to the original dataframe
df_refined['Wind_Chill(F)'] = df_windchill_fill['Wind_Chill(F)']
print(f"Entries with NaN Wind Chill: {df_refined['Wind_Chill(F)'].isna().sum()}")

In [ ]:
# deleting working df to free up memory
del df_windchill_fill
gc.collect()

In [ ]:
# replacing NaN values in the humidity column

def fill_missing_hum(df, date='Acc_date', zip='Zipcode', state='State', hum='Humidity(%)'):
    """
    Replaces NaN humidity values with the mean humidity of entries
    with the same date and zip code. If no match exists with date and county,
    uses date and state.
    """
    # Step 1: mean humidity for each date-zip combination
    df_hum_fill = df_refined.copy()
    
    hum_means_zip = df_hum_fill.groupby([date, zip])[hum].transform('mean')
    
    # fill NaN values with the date-zip group mean
    df_hum_fill[hum] = df_hum_fill[hum].fillna(hum_means_zip)
    
    # Step 2: date-state combination for remaining NaNs
    if df_hum_fill[hum].isna().any():
        remaining = df_hum_fill[hum].isna().sum()
        print(f"Info: {remaining} humidity entries still missing after date-zip fill.")
        print("Filling remaining with date-state mean.")
        
        hum_means_state = df_hum_fill.groupby([date, state])[hum].transform('mean')
        df_hum_fill[hum] = df_hum_fill[hum].fillna(hum_means_state)
    
    # Step 3: if any NaNs still remain, fill with overall mean as last resort
    if df_hum_fill[hum].isna().any():
        remaining = df_hum_fill[hum].isna().sum()
        print(f"Warning: {remaining} humidity entries still missing after date-state fill.")
        print("Filling remaining with overall mean as last resort.")
        df_hum_fill[hum] = df_hum_fill[hum].fillna(df_hum_fill[hum].mean())
    
    return df_hum_fill

df_hum_fill = fill_missing_hum(df_refined)

In [ ]:
# verifying NaNs have been replaced in the working df
df_hum_fill['Humidity(%)'].isna().sum()

In [ ]:
# writing NaN-free column back to the original dataframe
df_refined['Humidity(%)'] = df_hum_fill['Humidity(%)']
print(f"Entries with NaN humidity: {df_refined['Humidity(%)'].isna().sum()}")

In [ ]:
# deleting working df to free up memory
del df_hum_fill
gc.collect()

In [ ]:
# replacing NaN values in the pressure column

def fill_missing_press(df, date='Acc_date', zip='Zipcode', state='State', press='Pressure(in)'):
    """
    Replaces NaN pressure values with the mean pressure of entries
    with the same date and zip code. If no match exists with date and county,
    uses date and state.
    """
    # Step 1: mean pressure for each date-zip combination
    df_press_fill = df_refined.copy()
    
    press_means_zip = df_press_fill.groupby([date, zip])[press].transform('mean')
    
    # fill NaN values with the date-zip group mean
    df_press_fill[press] = df_press_fill[press].fillna(press_means_zip)
    
    # Step 2: date-state combination for remaining NaNs
    if df_press_fill[press].isna().any():
        remaining = df_press_fill[press].isna().sum()
        print(f"Info: {remaining} pressure entries still missing after date-zip fill.")
        print("Filling remaining with date-state mean.")
        
        press_means_state = df_press_fill.groupby([date, state])[press].transform('mean')
        df_press_fill[press] = df_press_fill[press].fillna(press_means_state)
    
    # Step 3: if any NaNs still remain, fill with overall mean as last resort
    if df_press_fill[press].isna().any():
        remaining = df_press_fill[press].isna().sum()
        print(f"Warning: {remaining} pressure entries still missing after date-state fill.")
        print("Filling remaining with overall mean as last resort.")
        df_press_fill[press] = df_press_fill[press].fillna(df_press_fill[press].mean())
    
    return df_press_fill

df_press_fill = fill_missing_press(df_refined)

In [ ]:
# verifying NaNs have been replaced in the working df
df_press_fill['Pressure(in)'].isna().sum()

In [ ]:
# writing NaN-free column back to the original dataframe
df_refined['Pressure(in)'] = df_press_fill['Pressure(in)']
print(f"Entries with NaN pressure: {df_refined['Pressure(in)'].isna().sum()}")

In [ ]:
# deleting working df to free up memory
del df_press_fill
gc.collect()

In [ ]:
# replacing NaN values in the visibility column

def fill_missing_vis(df, date='Acc_date', zip='Zipcode', state='State', vis='Visibility(mi)'):
    """
    Replaces NaN visibility values with the mean visibility of entries
    with the same date and zip code. If no match exists with date and county,
    uses date and state.
    """
    # Step 1: mean visibility for each date-zip combination
    df_vis_fill = df_refined.copy()
    
    vis_means_zip = df_vis_fill.groupby([date, zip])[vis].transform('mean')
    
    # fill NaN values with the date-zip group mean
    df_vis_fill[vis] = df_vis_fill[vis].fillna(vis_means_zip)
    
    # Step 2: date-state combination for remaining NaNs
    if df_vis_fill[vis].isna().any():
        remaining = df_vis_fill[vis].isna().sum()
        print(f"Info: {remaining} visibility entries still missing after date-zip fill.")
        print("Filling remaining with date-state mean.")
        
        vis_means_state = df_vis_fill.groupby([date, state])[vis].transform('mean')
        df_vis_fill[vis] = df_vis_fill[vis].fillna(vis_means_state)
    
    # Step 3: if any NaNs still remain, fill with overall mean as last resort
    if df_vis_fill[vis].isna().any():
        remaining = df_vis_fill[vis].isna().sum()
        print(f"Warning: {remaining} visibility entries still missing after date-state fill.")
        print("Filling remaining with overall mean as last resort.")
        df_vis_fill[vis] = df_vis_fill[vis].fillna(df_vis_fill[vis].mean())
    
    return df_vis_fill

df_vis_fill = fill_missing_vis(df_refined)

In [ ]:
# verifying NaNs have been replaced in the working df
df_vis_fill['Visibility(mi)'].isna().sum()

In [ ]:
# writing NaN-free column back to the original dataframe
df_refined['Visibility(mi)'] = df_vis_fill['Visibility(mi)']
print(f"Entries with NaN visibility: {df_refined['Visibility(mi)'].isna().sum()}")

In [ ]:
# deleting working df to free up memory
del df_vis_fill
gc.collect()

In [ ]:
# replacing NaN with 0 in wind speed and precipitation

df_refined['Wind_Speed(mph)'] = df_refined['Wind_Speed(mph)'].fillna(0)
df_refined['Precipitation(in)'] = df_refined['Precipitation(in)'].fillna(0)
print(f"Entries with NaN Wind Speed: {df_refined['Wind_Speed(mph)'].isna().sum()}")
print(f"Entries with NaN Precipitation: {df_refined['Precipitation(in)'].isna().sum()}")

In [ ]:
df_refined.columns

In [ ]:
# converting boolean to integer
df_refined['Amenity'] = df_refined['Amenity'].astype(int)
df_refined['Bump'] = df_refined['Bump'].astype(int)
df_refined['Crossing'] = df_refined['Crossing'].astype(int)
df_refined['Give_Way'] = df_refined['Give_Way'].astype(int)
df_refined['Junction'] = df_refined['Junction'].astype(int)
df_refined['No_Exit'] = df_refined['No_Exit'].astype(int)
df_refined['Railway'] = df_refined['Railway'].astype(int)
df_refined['Roundabout'] = df_refined['Roundabout'].astype(int)
df_refined['Station'] = df_refined['Station'].astype(int)
df_refined['Stop'] = df_refined['Stop'].astype(int)
df_refined['Traffic_Calming'] = df_refined['Traffic_Calming'].astype(int)
df_refined['Traffic_Signal'] = df_refined['Traffic_Signal'].astype(int)
df_refined['Turning_Loop'] = df_refined['Turning_Loop'].astype(int)

In [ ]:
# checking to see if the day/night data points are missing for the same entries
day_night_cols =['Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
blank_counts = df_refined[day_night_cols].isna().sum(axis=1)
results = blank_counts.value_counts().sort_index()
results

In [ ]:
del blank_counts
del results
gc.collect()

In [ ]:
# creating column displaying Day or Night based on a day with equal length days and nights
def day_or_night(t):
    if time(6, 0, 0) <= t < time(18, 0, 0):
        return 'Day'
    else:
        return 'Night'

df_refined['Day_Night_Calc'] = df_refined['Acc_time'].apply(day_or_night)
df_refined['Day_Night_Calc'].value_counts()

In [ ]:
df_refined_dup2 = df_refined.copy()

# replacing missing day/night values with the generic day/night column created above
for col in day_night_cols:
    df_refined_dup2[col] = np.where(df_refined_dup2[col].isna(), df_refined_dup2['Day_Night_Calc'], df_refined_dup2[col])

In [ ]:
# verifying that no empty entries remain
blank_counts2 = df_refined_dup2[day_night_cols].isna().sum(axis=1)
results2 = blank_counts2.value_counts().sort_index()
results2

In [ ]:
df_refined['Sunrise_Sunset'] = df_refined_dup2['Sunrise_Sunset']
df_refined['Civil_Twilight'] = df_refined_dup2['Civil_Twilight']
df_refined['Nautical_Twilight'] = df_refined_dup2['Nautical_Twilight']
df_refined['Astronomical_Twilight'] = df_refined_dup2['Astronomical_Twilight']

In [ ]:
del df_refined_dup2
del results2
gc.collect()

#### Cleaning Weather Condition

The Weather_Condition field has a large number of values across the dataset. Many of these values are used rarely and will complicate modeling, so I'll consolidate them into a smaller number of value options. This will be a manual process-I'll review and create a mapping table by hand.

In [ ]:
#print(f"Count of Unique Weather Condition Entries: {df_all_data['Weather_Condition'].nunique()}")
df_refined['Weather_Condition'].value_counts().sort_values(ascending=True)

In [ ]:
# printing a list of weather condition unique values to csv
unique_wthr_cond = pd.Series(df_refined['Weather_Condition'].unique())
unique_wthr_cond.to_csv('conditions.csv')

In [ ]:
def csv_to_dict_reader(filename):
    result = {}
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if len(row) >=2:
                result[row[0]] = row[1]
    return result

In [ ]:
weather_mapper = csv_to_dict_reader('conditions_consolidated.csv')
df_refined_dup1 = df_refined.copy()

df_refined_dup1['Weather_Condition'] = df_refined_dup1['Weather_Condition'].map(weather_mapper)
print(df_refined_dup1['Weather_Condition'].value_counts().sort_values(ascending=True))
print(df_refined_dup1['Weather_Condition'].nunique())

In [ ]:
df_refined['Weather_Condition'] = df_refined_dup1['Weather_Condition']
df_refined['Weather_Condition'].nunique()

In [ ]:
del df_refined_dup1
del weather_mapper
gc.collect()

### Constructing Data

I'm adding a feature that categorizes the location of the accidents as urban / suburban / rural. I'm using Rural-Urban Commuting Area Codes from the USDA Economic Research Service as the data source. https://www.ers.usda.gov/data-products/rural-urban-commuting-area-codes

In [ ]:
loc_type_mapper = csv_to_dict_reader('category_by_zip.csv')

In [ ]:
# converting zip code entries to the 5-digit format

df_refined['Zipcode'] = df_refined['Zipcode'].astype(str)
df_refined['Zipcode'] = df_refined['Zipcode'].str[:5]

# mapping a new location type field using the mapper created above
df_refined['Location_Type'] = df_refined['Zipcode'].map(loc_type_mapper)
df_refined['Location_Type'].isna().sum()

In [ ]:
# checking for clear pattern(s) in the missing location type rows
nan_zip_codes = df_refined.loc[df_refined['Location_Type'].isna(), 'Zipcode']
nan_zip_codes

In [ ]:
# determining proportions of location types in the whole dataset; will impute the NaNs with the same proportion
df_refined['Location_Type'].value_counts()

In [ ]:
# filling missing values in line with value distribution in the rest of the dataset
location_types = ['Urban', 'Suburban', 'Rural']
proportions = [0.83, 0.10, 0.07]

num_missing = df_refined['Location_Type'].isna().sum()
random_values = np.random.choice(location_types, size=num_missing, p=proportions)

df_refined.loc[df_refined['Location_Type'].isna(), 'Location_Type'] = random_values

In [ ]:
# verifying all rows have been filled
num_missing

In [ ]:
del nan_zip_codes
del num_missing
gc.collect()

In [ ]:
df_refined.columns

In [ ]:
df_clean = pd.read_csv('Data/cleaned.csv')

In [ ]:
# removing fields created for cleaning and used for imputation and construction 
df_cleaned = df_clean.drop(['Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'City', 'County', 'State', 'Zipcode', 'Distance(mi)'], axis=1)

In [ ]:
df_encoded = pd.get_dummies(df_cleaned, columns=['Weather_Condition', 'Sunrise_Sunset', 'Civil_Twilight', 
                                                 'Nautical_Twilight', 'Astronomical_Twilight', 'Location_Type'], drop_first=True)

In [ ]:
del df_cleaned
del df_clean
gc.collect()

In [ ]:
# setting X and y
X = df_encoded.drop(['Severity'], axis=1)
y = df_encoded['Severity']

# splitting data
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.2, random_state=42)

# scaling features in case I need something other than decision trees
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
del df_encoded
gc.collect()

## Analysis

Now that the data is preprocessed, we can move to the analysis. As mentioned earlier, I selected a random forest model in order to maximize the interpretability of the results. Interpretability is my priority because of the need to deliver clear busines insights; if the goal was to create an accurate predictive model, I would likely have chosen a different approach.

I'll start by creating a baseline model to provide a comparison point for optimization.

In [ ]:
# initializing a baseline model
base_model = DecisionTreeClassifier(random_state=42)

# cross-validation
base_scores = cross_val_score(base_model, X_train_scaled, 
                              y_train, cv=5, scoring='accuracy')

# fit baseline model to explore feature importance
base_model.fit(X_train_scaled, y_train)
initial_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': base_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Baseline model performance")
print(f"Accuracy: {base_scores.mean():.4f}(+/- {base_scores.std() * 2:.4f})")
print(f"Training Score: {base_model.score(X_train_scaled, y_train):.4f}")

print("\nInitial Feature Importance:")
print(initial_importance)

The baseline model is overfitting: its accuracy on training data is 96% vs. 74% on test (i.e. unseen) data.

I'll next look for optimal parameters for this model. I'm using a randomized search to efficiently explore a large parameter space, based on probability distributions around each parameter.

In [ ]:
gc.collect()

In [ ]:
import psutil

virtual_memory = psutil.virtual_memory()
available_gb = round(virtual_memory.available / (1024**3), 2)
available_gb

In [ ]:
# Define parameter distributions for random search
param_dist = {
   # uniform distribution from 1 to 20 for max_depth
   'max_depth': randint(1, 20),
  
   # log-uniform distribution from 2 to 50 for min_samples_split
   'min_samples_split': reciprocal(a=2, b=50).rvs(1000).round().astype(int),
  
   # log-uniform distribution from 1 to 20 for min_samples_leaf
   'min_samples_leaf': reciprocal(a=1, b=20).rvs(1000).round().astype(int),
  
   # categorical distribution for criterion
   'criterion': ['gini', 'entropy']
}

# Initialize randomized search
random_search = RandomizedSearchCV(
   DecisionTreeClassifier(random_state=42),
   param_distributions=param_dist,
   n_iter=100,  # Number of parameter settings sampled
   cv=5,
   scoring='accuracy',
   n_jobs=-1,
   random_state=42
)

# Perform random search
random_search.fit(X_train_scaled, y_train)


print("Best parameters from random search:")
print(random_search.best_params_)
print(f"Best cross-validation score: {random_search.best_score_:.3f}")
print(f"Best training score: {random_search.best_estimator_.score(X_train_scaled, y_train):.3f}")

I'll now perform a more focused parameter search using grid search. The best values from the random search inform the grid values below.

In [ ]:
# Adjust grid based on best parameters from random search
focused_param_grid = {
  'criterion': ['entropy', 'gini'],
   # For max_depth, we'll explore values around 14 (12-16)
  # The reasoning is that tree depth often has a sweet spot,
  # and we want to make sure 14 is truly optimal
  'max_depth': [4, 5, 6, 7, 8],
   # Since min_samples_leaf=2 was optimal, we'll explore very small values
  # This suggests the model benefits from granular splits
  'min_samples_leaf': [5, 7, 9],
   # For min_samples_split, we'll create a window around 46
  # We'll use smaller steps since we're in a promising region
  'min_samples_split': [5, 7, 9]
}


# Perform grid search
grid_search = GridSearchCV(
   DecisionTreeClassifier(random_state=42),
   focused_param_grid,
   cv=5,
   scoring='accuracy',
   n_jobs=-1
)


grid_search.fit(X_train_scaled, y_train)


print("Best parameters from grid search:")
print(grid_search.best_params_)
print(f"Best cross-validation score: {grid_search.best_score_:.3f}")
print(f"Best training score: {grid_search.best_estimator_.score(X_train_scaled, y_train):.3f}")

## Evaluation

### Business Insight/Recommendation 1

### Business Insight/Recommendation 2

### Business Insight/Recommendation 3

### Tableau Dashboard link

## Conclusion and Next Steps
Text here